# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4933, uptime 0:00:33
mini_beamline                    RUNNING   pid 4934, uptime 0:00:33
random_walk                      RUNNING   pid 4949, uptime 0:00:30
random_walk_horiz                RUNNING   pid 4936, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4937, uptime 0:00:33
simple                           RUNNING   pid 4938, uptime 0:00:33
thermo_sim                       RUNNING   pid 4939, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4940, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 13:03:55
Persistent Unique Scan ID: '023a664f-077f-4ffd-bdc5-50faf10df634'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 13:03:55.9 |     -1.000 |      99159 |


|         2 | 13:03:56.0 |     -0.500 |      99389 |
|         3 | 13:03:56.0 |      0.000 |      99626 |
|         4 | 13:03:56.0 |      0.500 |      99479 |
|         5 | 13:03:56.0 |      1.000 |      97804 |
+-----------+------------+------------+------------+


generator scan ['023a664f'] (scan num: 3)





('023a664f-077f-4ffd-bdc5-50faf10df634',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/023a664f-077f-4ffd-bdc5-50faf10df634-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 13:03:56
Persistent Unique Scan ID: 'f9a7190d-1bc6-4f3b-8f3b-da9d0740a3ac'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 13:03:56.5 |          -1 | 1068276808 |


|         2 | 13:03:56.6 |          -0 | 1063818995 |


|         3 | 13:03:56.7 |           0 | 1057801473 |
|         4 | 13:03:56.8 |           0 | 1054900040 |


|         5 | 13:03:56.9 |           1 | 1053546424 |


+-----------+------------+-------------+------------+
generator scan ['f9a7190d'] (scan num: 4)





('f9a7190d-1bc6-4f3b-8f3b-da9d0740a3ac',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 13:03:57
Persistent Unique Scan ID: '06b89a14-c4d3-4312-94e2-f2202715a379'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 13:03:57.4 | 1058179014 |


|         2 | 13:03:58.3 | 1113560402 |


|         3 | 13:03:59.3 | 1125424226 |


...

|        28 | 13:04:24.4 | 1073249502 |


|        29 | 13:04:25.4 | 1060897593 |


|        30 | 13:04:26.4 | 1116080437 |


+-----------+------------+------------+
generator count ['06b89a14'] (scan num: 5)





('06b89a14-c4d3-4312-94e2-f2202715a379',)